# 🤖 Multi-Agent ATS Resume Analyzer System

**A Production-Ready Multi-Agent System for ATS Resume Screening**

This notebook implements a complete **Applicant Tracking System (ATS)** powered by **8 specialized AI agents** that work together to analyze, score, and provide feedback on resumes against job descriptions.

### How It Works
Each agent has a distinct role and personality — from deterministic extractors to semantic reasoners — mimicking how a real hiring pipeline operates:

| # | Agent | Type | Role |
|---|-------|------|------|
| 1 | **CV Parsing Agent** | Extractor (Tool-augmented) | Extracts structured data (skills, experience, education) from PDF/DOCX/TXT resumes |
| 2 | **Job Description Understanding Agent** | Semantic Analyst | Analyzes job postings to extract must-have/nice-to-have skills using NLP + knowledge base |
| 3 | **Keyword Matching Agent** | Rule-Based Evaluator | Simulates traditional ATS keyword scoring with weighted sub-scores |
| 4 | **Semantic Similarity Agent** | Semantic Reasoning | Goes beyond keywords — uses sentence embeddings to find synonym matches |
| 5 | **Formatting Validator Agent** | Validator / Inspector | Checks for ATS-unfriendly formatting (tables, columns, images, icons) |
| 6 | **Bias & Compliance Agent** | Policy / Safety | Flags age, gender, nationality info to ensure ethical screening |
| 7 | **Feedback & Rewrite Agent** | Generative Coach | Provides actionable bullet-point suggestions for missing skills |
| 8 | **Orchestrator Agent** | Manager | Coordinates all agents, resolves conflicts, and produces the final report |

### Tech Stack
- **NLP**: spaCy (`en_core_web_sm`) for entity recognition and text processing
- **Embeddings**: Sentence-Transformers (`all-MiniLM-L6-v2`) for semantic similarity
- **PDF Parsing**: PyPDF2 + pdfplumber for text and visual layout extraction
- **API**: FastAPI + ngrok for production deployment

---

## 📦 Installation & Setup

Install all required Python packages and download the spaCy English language model. These are the core dependencies:
- **PyPDF2** / **python-docx**: Parse PDF and DOCX resume files
- **spaCy**: NLP pipeline for tokenization, entity recognition, and text processing
- **sentence-transformers**: Pre-trained transformer models for computing semantic similarity between texts
- **openai** / **anthropic**: (Optional) LLM API clients for future generative enhancements

In [ ]:
# Install required packages:
# - PyPDF2: Reads text from PDF files
# - python-docx: Reads text from Microsoft Word (.docx) files
# - spacy: Industrial-strength NLP library for text processing
# - sentence-transformers: Provides pre-trained models for computing dense vector embeddings
# - openai, anthropic: Optional API clients for LLM-based enhancements
!pip install -q PyPDF2 python-docx spacy sentence-transformers openai anthropic

# Download the small English language model for spaCy (tokenization, POS tagging, NER)
!python -m spacy download en_core_web_sm

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 253.0/253.0 kB 17.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 405.9/405.9 kB 27.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 77.6 MB/s eta 0:00:0000:010:01
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [ ]:
# ============================================================================
# IMPORT LIBRARIES & LOAD NLP MODELS
# ============================================================================
# This cell imports all core libraries used throughout the notebook and
# initializes two global NLP models that are shared across multiple agents:
#   1. spaCy 'en_core_web_sm' — used for tokenization, part-of-speech tagging,
#      and named entity recognition in the CV Parsing Agent.
#   2. SentenceTransformer 'all-MiniLM-L6-v2' — a lightweight transformer model
#      that encodes text into 384-dim vectors for cosine similarity comparisons.
#      Used by the JD Agent, ATS Scorer, Semantic Agent, and Feedback Agent.
# ============================================================================

import re                                          # Regex for pattern extraction (emails, phones, skills)
import json                                        # JSON serialization for report output
import PyPDF2                                      # PDF text extraction
import docx                                        # DOCX text extraction
import spacy                                       # NLP pipeline (tokenization, NER)
from sentence_transformers import SentenceTransformer, util  # Semantic similarity via embeddings
from typing import Dict, List, Tuple, Optional     # Type hints for function signatures
from dataclasses import dataclass, asdict          # Structured data containers (auto __init__, __repr__)
from enum import Enum                              # Enumeration for decision outcomes (PASS/BORDERLINE/REJECT)
import warnings
warnings.filterwarnings('ignore')                  # Suppress transformer/deprecation warnings

# Load NLP models (these are used globally by multiple agents)
nlp = spacy.load('en_core_web_sm')                            # spaCy small English model
embedding_model = SentenceTransformer('all-MiniLM-L6-v2')     # Sentence embedding model (384-dim vectors)

print("✅ All libraries loaded successfully")

2026-02-14 00:14:24.461043: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1771028064.676212      55 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1771028064.742896      55 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1771028065.314634      55 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1771028065.314689      55 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1771028065.314692      55 computation_placer.cc:177] computation placer alr

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

✅ All libraries loaded successfully


## 📋 Data Structures

This cell defines all the **dataclasses** (structured data containers) used to pass information between agents. Each dataclass represents a specific piece of the analysis pipeline:

| Dataclass | Purpose |
|-----------|---------|
| `PersonalInfo` | Stores contact details extracted from the CV (name, email, phone, location, LinkedIn) |
| `Experience` | A single work experience entry (title, company, duration, bullet descriptions) |
| `Education` | A single education entry (degree, institution, year, GPA) |
| `ParsedCV` | The complete structured output of Agent 1 — all extracted CV data |
| `JobRequirements` | The structured output of Agent 2 — extracted job requirements |
| `ATSScore` | The scoring output of Agent 3 — overall + sub-scores with breakdowns |
| `SemanticAnalysis` | The output of Agent 4 — similarity scores and semantic skill matches |
| `FormatIssues` | The output of Agent 5 — ATS compatibility flags and recommendations |
| `BiasFlags` | The output of Agent 6 — detected bias indicators and compliance status |
| `Feedback` | The output of Agent 7 — suggested rewrites, missing keywords, priorities |
| `Decision` | Enum with three outcomes: `PASS`, `BORDERLINE`, `REJECT` |
| `FinalReport` | The combined output of Agent 8 — aggregates all agent outputs into one report |

In [ ]:
# ============================================================================
# DATA MODELS (DATACLASSES)
# ============================================================================
# These dataclasses define the schema for all inter-agent communication.
# Using @dataclass gives us automatic __init__, __repr__, and asdict() support,
# making it easy to serialize results to JSON for the API.
# ============================================================================

# --- CV Extraction Models ---

@dataclass
class PersonalInfo:
    """Contact details extracted from the resume header."""
    name: str = ""          # Candidate's full name
    email: str = ""         # Email address (extracted via regex)
    phone: str = ""         # Phone number (extracted via regex)
    location: str = ""      # City/State/Country
    linkedin: str = ""      # LinkedIn profile URL

@dataclass
class Experience:
    """A single work experience entry from the CV."""
    title: str              # Job title (e.g., "Senior Software Engineer")
    company: str            # Company name
    duration: str           # Date range (e.g., "2021 - Present")
    description: List[str]  # Bullet-point descriptions of responsibilities/achievements

@dataclass
class Education:
    """A single education entry from the CV."""
    degree: str             # Degree name (e.g., "B.S. in Computer Science")
    institution: str        # University/college name
    year: str               # Graduation year
    gpa: str = ""           # GPA (optional)

@dataclass
class ParsedCV:
    """Complete structured output from the CV Parsing Agent (Agent 1)."""
    personal_info: PersonalInfo     # Contact information
    skills: List[str]               # List of extracted technical/soft skills
    experience: List[Experience]    # Work experience entries
    education: List[Education]      # Education entries
    projects: List[str]             # Project descriptions (if found)
    certifications: List[str]       # Certifications (if found)
    raw_text: str                   # Cleaned/normalized full text of the CV

# --- Job Description Models ---

@dataclass
class JobRequirements:
    """Structured output from the Job Description Agent (Agent 2)."""
    must_have_skills: List[str]     # Required/mandatory skills
    nice_to_have_skills: List[str]  # Preferred/bonus skills
    years_of_experience: int        # Minimum years of experience required
    tools_technologies: List[str]   # Combined list of all tools/technologies
    soft_skills: List[str]          # Soft skills mentioned (communication, teamwork, etc.)
    job_title: str                  # Job title from the posting
    raw_text: str                   # Original job description text

# --- Scoring & Analysis Models ---

@dataclass
class ATSScore:
    """Scoring output from the ATS Scoring Agent (Agent 3)."""
    overall_score: float                # Weighted aggregate score (0-100)
    skill_match_score: float            # % of must-have skills matched
    experience_score: float             # Experience years vs. requirement
    title_similarity_score: float       # Semantic similarity between CV titles and JD title
    missing_must_have: List[str]        # Must-have skills NOT found in the CV
    matched_skills: List[str]           # All skills successfully matched
    breakdown: Dict[str, float]         # Detailed sub-score breakdown

@dataclass
class SemanticAnalysis:
    """Output from the Semantic Similarity Agent (Agent 4)."""
    cv_jd_similarity: float                             # Overall cosine similarity between CV and JD text
    skill_semantic_matches: Dict[str, List[Tuple[str, float]]]  # JD skill → [(CV skill, similarity score)]
    experience_relevance: float                         # How relevant CV experience is to JD responsibilities

# --- Validation & Compliance Models ---

@dataclass
class FormatIssues:
    """Output from the Formatting Validator Agent (Agent 5)."""
    has_tables: bool                # Whether the document contains tables
    has_columns: bool               # Whether a multi-column layout was detected
    has_images: bool                # Whether images/graphics are embedded
    has_icons: bool                 # Whether special Unicode icons/emojis are present
    parsing_risks: List[str]        # List of identified ATS parsing risks
    ats_friendly: bool              # Overall ATS compatibility verdict
    recommendations: List[str]     # Actionable recommendations to fix issues

@dataclass
class BiasFlags:
    """Output from the Bias & Compliance Agent (Agent 6)."""
    age_indicators: List[str]           # Detected age-related info (birth year, "years old")
    gender_bias: List[str]              # Detected gender-specific language
    nationality_references: List[str]   # Detected nationality/ethnicity references
    photo_detected: bool                # Whether a photo/headshot was found
    compliance_issues: List[str]        # Summarized compliance problems
    is_compliant: bool                  # Overall compliance verdict

# --- Feedback & Decision Models ---

@dataclass
class Feedback:
    """Output from the Feedback & Rewrite Agent (Agent 7)."""
    skill_rewording: Dict[str, str]     # Original skill → suggested rewording
    stronger_bullets: List[str]         # Suggestions to strengthen weak bullet points
    missing_keywords: List[str]         # Keywords missing from the CV
    rewritten_sections: Dict[str, str]  # Section name → rewritten content
    improvement_priority: List[str]     # Prioritized list of improvements

class Decision(Enum):
    """Final screening decision — used by the Orchestrator Agent."""
    PASS = "pass"               # Candidate meets requirements → proceed to interview
    BORDERLINE = "borderline"   # Partial match → needs manual review
    REJECT = "reject"           # Insufficient match → does not meet minimum requirements

@dataclass
class FinalReport:
    """Combined output from the Orchestrator Agent (Agent 8) — the final deliverable."""
    decision: Decision                      # PASS / BORDERLINE / REJECT
    ats_score: ATSScore                     # Agent 3 output
    semantic_analysis: SemanticAnalysis      # Agent 4 output
    format_issues: FormatIssues             # Agent 5 output
    bias_flags: BiasFlags                   # Agent 6 output
    feedback: Feedback                      # Agent 7 output
    improvement_checklist: List[str]        # Aggregated actionable checklist
    summary: str                            # Executive summary string

print("✅ Data structures defined")

✅ Data structures defined


## 🤖 Agent 1: CV Parsing Agent (Extractor Agent)

**Agent Type:** Tool-augmented Extractor  
**Role:** Extract structured data from resumes (PDF, DOCX, TXT)  
**Characteristics:** Deterministic, high accuracy, no creativity — purely rule-based extraction

This agent is the **entry point** of the pipeline. It takes a raw resume file and outputs a `ParsedCV` dataclass containing:
- **Personal Info**: Email and phone extracted via regex patterns
- **Skills**: Identified using a keyword list + regex-based skills section parsing
- **Experience**: Job titles, companies, durations, and bullet descriptions parsed from the "Experience" section
- **Education**: Degree, institution, and graduation year extracted from the "Education" section
- **Normalized Text**: Cleaned text with bullets, icons, and extra whitespace removed

**Key Methods:**
- `parse_pdf()` / `parse_docx()` — File format handlers using PyPDF2 and python-docx
- `extract_email()` / `extract_phone()` — Regex-based contact extraction
- `extract_skills()` — Combines keyword matching with skills-section parsing
- `extract_experience()` / `extract_education()` — Section-based structured extraction
- `normalize_text()` — Strips formatting artifacts for downstream processing

In [4]:
class CVParsingAgent:
    """
    Agent Type: Extractor Agent (Tool-augmented)
    Role: Extract structured data from CV
    Characteristics: Deterministic, high accuracy, no creativity
    """
    
    def __init__(self):
        self.nlp = nlp
    
    def parse_pdf(self, pdf_path: str) -> str:
        """Extract text from PDF"""
        text = ""
        try:
            with open(pdf_path, 'rb') as file:
                pdf_reader = PyPDF2.PdfReader(file)
                for page in pdf_reader.pages:
                    text += page.extract_text()
        except Exception as e:
            print(f"Error parsing PDF: {e}")
        return text
    
    def parse_docx(self, docx_path: str) -> str:
        """Extract text from DOCX"""
        text = ""
        try:
            doc = docx.Document(docx_path)
            for paragraph in doc.paragraphs:
                text += paragraph.text + "\n"
        except Exception as e:
            print(f"Error parsing DOCX: {e}")
        return text
    
    def extract_email(self, text: str) -> str:
        """Extract email using regex"""
        email_pattern = r'\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,}\b'
        emails = re.findall(email_pattern, text)
        return emails[0] if emails else ""
    
    def extract_phone(self, text: str) -> str:
        """Extract phone number"""
        phone_pattern = r'\+?\d[\d\s\-\(\)]{8,}\d'
        phones = re.findall(phone_pattern, text)
        return phones[0] if phones else ""
    
    def extract_skills(self, text: str) -> List[str]:
        """Extract skills using pattern matching and NLP"""
        # Common skill indicators
        skill_keywords = [
            'python', 'java', 'javascript', 'c\+\+', 'sql', 'nosql',
            'machine learning', 'deep learning', 'nlp', 'computer vision',
            'tensorflow', 'pytorch', 'keras', 'scikit-learn',
            'aws', 'azure', 'gcp', 'docker', 'kubernetes',
            'react', 'angular', 'vue', 'node.js', 'express',
            'git', 'ci/cd', 'agile', 'scrum'
        ]
        
        skills = []
        text_lower = text.lower()
        
        for skill in skill_keywords:
            if re.search(r'\b' + skill + r'\b', text_lower):
                skills.append(skill.title())
        
        # Look for skills section
        skills_section = re.search(r'skills?[:\n](.*?)(?=\n[A-Z]|$)', text, re.IGNORECASE | re.DOTALL)
        if skills_section:
            section_text = skills_section.group(1)
            # Extract comma-separated or bullet-pointed skills
            extracted = re.findall(r'[•●○-]?\s*([A-Za-z][A-Za-z\s\.\+#]{2,}?)(?:[,;•●○]|\n|$)', section_text)
            skills.extend([s.strip() for s in extracted if len(s.strip()) > 2])
        
        return list(set(skills))  # Remove duplicates
    
    def extract_experience(self, text: str) -> List[Experience]:
        """Extract work experience"""
        experiences = []
        
        # Find experience section
        exp_section = re.search(r'experience[:\n](.*?)(?=education|projects|skills|$)', text, re.IGNORECASE | re.DOTALL)
        if not exp_section:
            return experiences
        
        section_text = exp_section.group(1)
        
        # Pattern for job entries (simplified)
        job_pattern = r'([A-Z][^\n]{5,60})\s*[\n|]\s*([A-Z][^\n]{5,60})\s*[\n|]\s*([\d]{4}.*?[\d]{4}|present)'
        matches = re.finditer(job_pattern, section_text, re.IGNORECASE)
        
        for match in matches:
            title = match.group(1).strip()
            company = match.group(2).strip()
            duration = match.group(3).strip()
            
            # Extract bullet points after this job
            start_pos = match.end()
            next_job = re.search(job_pattern, section_text[start_pos:], re.IGNORECASE)
            end_pos = start_pos + next_job.start() if next_job else len(section_text)
            
            job_desc = section_text[start_pos:end_pos]
            bullets = re.findall(r'[•●○-]\s*(.+)', job_desc)
            
            experiences.append(Experience(
                title=title,
                company=company,
                duration=duration,
                description=bullets[:5]  # Limit to 5 bullets
            ))
        
        return experiences
    
    def extract_education(self, text: str) -> List[Education]:
        """Extract education"""
        education = []
        
        # Find education section
        edu_section = re.search(r'education[:\n](.*?)(?=experience|projects|skills|$)', text, re.IGNORECASE | re.DOTALL)
        if not edu_section:
            return education
        
        section_text = edu_section.group(1)
        
        # Pattern for education entries
        edu_pattern = r'(bachelor|master|phd|b\.?s\.?|m\.?s\.?|ph\.?d\.?)[^\n]{5,80}'
        matches = re.finditer(edu_pattern, section_text, re.IGNORECASE)
        
        for match in matches:
            degree_line = match.group(0).strip()
            
            # Extract year
            year_match = re.search(r'(19|20)\d{2}', degree_line)
            year = year_match.group(0) if year_match else "Unknown"
            
            education.append(Education(
                degree=degree_line,
                institution="Unknown",  # Could be improved with NER
                year=year
            ))
        
        return education
    
    def normalize_text(self, text: str) -> str:
        """Remove formatting, bullets, icons"""
        # Remove bullet points
        text = re.sub(r'[•●○■□▪▫◆◇]', '', text)
        # Remove multiple spaces
        text = re.sub(r'\s+', ' ', text)
        # Remove special characters except basic punctuation
        text = re.sub(r'[^\w\s,.;:!?@#()\-]', '', text)
        return text.strip()
    
    def parse(self, file_path: str) -> ParsedCV:
        """Main parsing method"""
        print(f"🔍 Parsing CV: {file_path}")
        
        # Extract text based on file type
        if file_path.endswith('.pdf'):
            raw_text = self.parse_pdf(file_path)
        elif file_path.endswith('.docx'):
            raw_text = self.parse_docx(file_path)
        else:
            with open(file_path, 'r', encoding='utf-8') as f:
                raw_text = f.read()
        
        # Extract structured data
        personal_info = PersonalInfo(
            email=self.extract_email(raw_text),
            phone=self.extract_phone(raw_text)
        )
        
        skills = self.extract_skills(raw_text)
        experience = self.extract_experience(raw_text)
        education = self.extract_education(raw_text)
        
        normalized_text = self.normalize_text(raw_text)
        
        parsed_cv = ParsedCV(
            personal_info=personal_info,
            skills=skills,
            experience=experience,
            education=education,
            projects=[],  # Could be extended
            certifications=[],  # Could be extended
            raw_text=normalized_text
        )
        
        print(f"✅ Extracted {len(skills)} skills, {len(experience)} experiences, {len(education)} education entries")
        return parsed_cv

# Test the agent
print("✅ CV Parsing Agent initialized")

✅ CV Parsing Agent initialized


## 🤖 Agent 2: Job Description Understanding Agent (Analyst Agent)

In [5]:
class JobDescriptionAgent:
    """
    Agent Type: Semantic Analyst Agent
    Role: Extract ONLY real technical skills using Knowledge Base + Semantic Validation
    """
    
    def __init__(self):
        self.nlp = nlp
        # The "Brain": Use the embedding model defined globally
        self.model = embedding_model 
        
        # 1. KNOWLEDGE BASE: Known technical skills (Whitelist)
        self.known_skills = {
            'python', 'java', 'c++', 'javascript', 'typescript', 'html', 'css', 'sql', 'nosql',
            'react', 'angular', 'vue', 'node.js', 'django', 'flask', 'fastapi', 'spring',
            'docker', 'kubernetes', 'aws', 'azure', 'gcp', 'terraform', 'ansible', 'jenkins',
            'machine learning', 'deep learning', 'nlp', 'computer vision', 'pytorch', 'tensorflow',
            'keras', 'scikit-learn', 'pandas', 'numpy', 'matplotlib', 'seaborn', 'opencv',
            'git', 'github', 'gitlab', 'jira', 'agile', 'scrum', 'linux', 'unix', 'bash',
            'redis', 'mongodb', 'postgresql', 'mysql', 'oracle', 'elasticsearch', 'kafka',
            'spark', 'hadoop', 'hive', 'tableau', 'power bi', 'figma', 'llm', 'llms',
            'transformers', 'huggingface', 'langchain', 'mlflow', 'mlops', 'ci/cd', 'rest api',
            'graphql', 'grpc', 'bert', 'gpt', 'llama', 'generative ai', 'rag'
        }

        # 2. NOISE BLOCKLIST: Words that often get mistaken for skills
        self.blocklist = {
            'essential', 'required', 'mandatory', 'must', 'have', 'nice', 'preferred', 'desired',
            'plus', 'strong', 'excellent', 'good', 'ability', 'proficient', 'experience', 'years',
            'skills', 'knowledge', 'background', 'familiarity', 'hands-on', 'hands', 'expert',
            'role', 'position', 'responsibilities', 'qualifications', 'requirements', 'about',
            'team', 'work', 'environment', 'communication', 'collaboration', 'problem', 'solving',
            'analytical', 'thinking', 'degree', 'computer', 'science', 'engineering', 'complex',
            'concepts', 'technical', 'fast-paced', 'paced', 'tier', 'conferences', 'description',
            'learning', 'frameworks', 'tools', 'platforms', 'apis', 'capability', 'like', 'with',
            'and', 'or', 'for', 'the', 'soft', 'tuning', 'deployment', 'production', 'training',
            'inference', 'fine-tuning', 'development', 'design', 'architecture', 'implementation',
            'support', 'maintenance', 'testing', 'debugging', 'optimization', 'performance',
            'scalability', 'security', 'privacy', 'compliance', 'regulations', 'standards',
            'best practices', 'methodologies', 'processes', 'procedures', 'documentation',
            'reporting', 'presentation', 'meetings', 'deadlines', 'priorities', 'stakeholders',
            'clients', 'customers', 'users', 'business', 'requirements', 'needs', 'goals',
            'objectives', 'strategies', 'tactics', 'plans', 'roadmaps', 'timelines', 'budgets',
            'resources', 'costs', 'risks', 'issues', 'challenges', 'opportunities', 'trends',
            'insights', 'analytics', 'data', 'information', 'systems', 'applications',
            'software', 'hardware', 'networks', 'servers', 'databases', 'cloud', 'infrastructure',
            'services', 'solutions', 'products', 'projects', 'programs', 'portfolios',
            'operations', 'management', 'leadership', 'mentorship', 'coaching', 'training',
            'hiring', 'recruiting', 'onboarding', 'performance', 'reviews', 'feedback',
            'career', 'growth', 'development', 'salary', 'compensation', 'benefits', 'perks',
            'culture', 'values', 'mission', 'vision', 'purpose', 'impact', 'innovation',
            'creativity', 'collaboration', 'teamwork', 'communication', 'transparency',
            'integrity', 'accountability', 'ownership', 'empowerment', 'diversity', 'inclusion',
            'equity', 'belonging', 'wellness', 'balance', 'flexibility', 'remote', 'hybrid',
            'office', 'location', 'relocation', 'travel', 'visa', 'sponsorship', 'citizenship',
            'residency', 'authorization', 'clearance', 'background', 'check', 'drug', 'test',
            'education', 'certification', 'license', 'degree', 'university', 'college', 'school',
            'course', 'bootcamp', 'workshop', 'seminar', 'conference', 'icml', 'neurips', 'cvpr',
            'iclr', 'aaai', 'ijcai', 'emnlp', 'naacl', 'eacl', 'acl', 'kdd', 'sigmod', 'vldb',
            'icde', 'www', 'sigir', 'cikm', 'wsdm', 'recsys', 'chi', 'uist', 'cscw', 'hri',
            'iros', 'icra', 'rss', 'corl', 'aistats', 'uai', 'colt', 'focs', 'stoc', 'soda',
            'crypto', 'eurocrypt', 'asiacrypt', 'ches', 'sp', 'ccs', 'usenix', 'ndss', 'osdi',
            'sosp', 'nsdi', 'sigcomm', 'mobicom', 'mobisys', 'infocom', 'globecom', 'icc',
            'fast', 'of'
        }
        
        # 3. Anchor embeddings for semantic validation
        # We compare unknown words to these concepts. If similar, we keep them.
        self.tech_anchors = ["software technology tool", "programming language framework", "computer library"]
        self.anchor_embeddings = self.model.encode(self.tech_anchors)

    def is_semantic_match(self, term: str, threshold: float = 0.35) -> bool:
        """Ask the AI: 'Is this word similar to a software tool?'"""
        try:
            term_emb = self.model.encode(term)
            # Calculate similarity with anchors
            scores = util.cos_sim(term_emb, self.anchor_embeddings)
            max_score = float(scores.max())
            
            # If high similarity, it's likely a tech skill
            return max_score > threshold
        except:
            return False

    def validate_candidate(self, candidate: str) -> bool:
        """The Gatekeeper: Decide if a word is a real skill"""
        clean_term = candidate.lower().strip().strip('.,()[]:')
        
        # Rule 1: Blocklist (Instant Fail)
        if clean_term in self.blocklist or len(clean_term) < 2:
            return False
            
        # Rule 2: Whitelist (Instant Pass)
        if clean_term in self.known_skills:
            return True
            
        # Rule 3: Heuristics (Regex patterns for versions/acronyms)
        # e.g., "GPT-4", "Llama-2", "C#"
        if re.search(r'\d', clean_term) or clean_term in ['c#', '.net']:
            return True
            
        # Rule 4: Semantic AI Check (The "Maybe" Pile)
        # If it's a capitalized word not in blocklist, ask the AI
        if candidate[0].isupper():
            return self.is_semantic_match(clean_term)
            
        return False

    def extract_skills_from_jd(self, text: str) -> Tuple[List[str], List[str]]:
        must_have = []
        nice_to_have = []
        
        # Find requirements section
        req_section = re.search(r'(requirements?|qualifications?|skills?)[:\n](.*?)(?=responsibilities|about|$)', 
                                text, re.IGNORECASE | re.DOTALL)
        target_text = req_section.group(2) if req_section else text
        
        # Split by lines
        lines = re.split(r'[\n•●○-]', target_text)
        
        for line in lines:
            # Skip empty lines
            if not line.strip(): continue
            
            # Detect category (Must have vs Nice to have)
            line_lower = line.lower()
            is_nice = any(x in line_lower for x in ['preferred', 'plus', 'bonus', 'nice'])
            
            # 1. Extract potential candidates using Regex
            # Captures: Acronyms (AWS), Capitalized Words (Python), and common Tech terms
            candidates = re.findall(r'\b[A-Z][a-zA-Z0-9\+\-\.]{1,}\b|\b(?:python|java|sql|react|node|docker|linux)\b', line)
            
            for cand in candidates:
                # 2. Validate every candidate
                if self.validate_candidate(cand):
                    if is_nice:
                        nice_to_have.append(cand)
                    else:
                        must_have.append(cand)

        return list(set(must_have)), list(set(nice_to_have))

    # Keep these standard helper methods
    def extract_years_of_experience(self, text: str) -> int:
        match = re.search(r'(\d+)\+?\s*(?:-\s*\d+)?\s*years?', text, re.IGNORECASE)
        return int(match.group(1)) if match else 0

    def extract_job_title(self, text: str) -> str:
        lines = text.strip().split('\n')
        return lines[0].strip() if lines else "Unknown"

    def extract_soft_skills(self, text: str) -> List[str]:
        # Simple keyword match for soft skills
        keywords = ['communication', 'teamwork', 'problem solving', 'leadership']
        return [k.title() for k in keywords if k in text.lower()]

    def analyze(self, jd_text: str) -> JobRequirements:
        print("🔍 Analyzing Job Description with Semantic Agent...")
        must, nice = self.extract_skills_from_jd(jd_text)
        years = self.extract_years_of_experience(jd_text)
        
        return JobRequirements(
            must_have_skills=must,
            nice_to_have_skills=nice,
            years_of_experience=years,
            tools_technologies=must + nice,
            soft_skills=self.extract_soft_skills(jd_text),
            job_title=self.extract_job_title(jd_text),
            raw_text=jd_text
        )

# Re-initialize the agent
print("✅ Semantic Job Agent Initialized")
agent = JobDescriptionAgent()

✅ Semantic Job Agent Initialized


## 🤖 Agent 3: Keyword Matching & ATS Scoring Agent (Rule-Based Evaluator)

In [6]:
class ATSScoringAgent:
    """
    Agent Type: Rule-Based Evaluator Agent
    Role: Simulate ATS filtering logic with improved Semantic Title Matching
    """
    
    def __init__(self):
        # We use the global embedding model for better title similarity
        self.model = embedding_model 
        
        # Scoring weights
        self.weights = {
            'must_have_match': 0.45,    # Increased weight for core skills
            'nice_to_have_match': 0.10,
            'experience_years': 0.25,
            'title_similarity': 0.20
        }
    
    def calculate_skill_match(self, cv_skills: List[str], required_skills: List[str]) -> Tuple[float, List[str], List[str]]:
        if not required_skills:
            return 0.0, [], [] 
        
        cv_skills_lower = [s.lower() for s in cv_skills]
        required_lower = [s.lower() for s in required_skills]
        
        matched = []
        missing = []
        
        for req_skill in required_lower:
            # Check for exact match or substring match (e.g. "React" in "React.js")
            if any(req_skill in cv_s or cv_s in req_skill for cv_s in cv_skills_lower):
                matched.append(req_skill)
            else:
                missing.append(req_skill)
        
        match_percentage = len(matched) / len(required_lower)
        return match_percentage, matched, missing
    
    def calculate_experience_score(self, cv_experience: List[Experience], required_years: int) -> float:
        if required_years == 0:
            return 1.0
        
        total_years = 0
        for exp in cv_experience:
            # Smart check for years in string duration
            years_match = re.search(r'(\d+)\s*year', exp.duration, re.IGNORECASE)
            if years_match:
                total_years += int(years_match.group(1))
            elif "present" in exp.duration.lower() or "current" in exp.duration.lower():
                total_years += 2 # Assume 2 years if it's a current ongoing role
            else:
                total_years += 1
        
        return min(1.0, total_years / required_years)
    
    def calculate_title_similarity(self, cv_titles: List[str], jd_title: str) -> float:
        """NEW: Uses AI Brain to compare titles instead of just counting words"""
        if not cv_titles or not jd_title:
            return 0.5
        
        # Encode the Job Title once
        jd_emb = self.model.encode(jd_title.lower())
        
        max_sim = 0
        for cv_title in cv_titles:
            cv_emb = self.model.encode(cv_title.lower())
            # Calculate cosine similarity between titles
            sim = float(util.cos_sim(jd_emb, cv_emb))
            max_sim = max(max_sim, sim)
            
        return max_sim

    def detect_keyword_stuffing(self, cv_text: str, keywords: List[str]) -> bool:
        cv_lower = cv_text.lower()
        for keyword in keywords:
            if cv_lower.count(keyword.lower()) > 6:
                return True
        return False
    
    def score(self, parsed_cv: ParsedCV, job_req: JobRequirements) -> ATSScore:
        print("📊 Calculating ATS Score (Semantic Title Mode)...")
        
        # 1-4. Calculate sub-scores
        must_have_score, matched_must, missing_must = self.calculate_skill_match(parsed_cv.skills, job_req.must_have_skills)
        nice_have_score, matched_nice, _ = self.calculate_skill_match(parsed_cv.skills, job_req.nice_to_have_skills)
        exp_score = self.calculate_experience_score(parsed_cv.experience, job_req.years_of_experience)
        
        cv_titles = [exp.title for exp in parsed_cv.experience]
        title_score = self.calculate_title_similarity(cv_titles, job_req.job_title)
        
        # 5. Weighted aggregation
        overall = (
            must_have_score * self.weights['must_have_match'] +
            nice_have_score * self.weights['nice_to_have_match'] +
            exp_score * self.weights['experience_years'] +
            title_score * self.weights['title_similarity']
        )
        
        # 6. Keyword Stuffing Penalty
        if self.detect_keyword_stuffing(parsed_cv.raw_text, job_req.must_have_skills):
            overall *= 0.8
            print("⚠️ Keyword stuffing detected - Penalty applied")
        
        return ATSScore(
            overall_score=round(overall * 100, 2),
            skill_match_score=round(must_have_score * 100, 2),
            experience_score=round(exp_score * 100, 2),
            title_similarity_score=round(title_score * 100, 2),
            missing_must_have=missing_must,
            matched_skills=matched_must + matched_nice,
            breakdown={
                'must_have': must_have_score * 100,
                'nice_have': nice_have_score * 100,
                'experience': exp_score * 100,
                'title': title_score * 100
            }
        )

print("✅ ATS Scoring Agent Updated with Semantic Title Matching")

✅ ATS Scoring Agent Updated with Semantic Title Matching


## 🤖 Agent 4: Semantic Similarity Agent (Semantic Reasoning Agent)

In [7]:
class SemanticSimilarityAgent:
    """
    Agent Type: Semantic Reasoning Agent
    Role: Go beyond keyword matching
    Characteristics: Uses embeddings, handles synonyms, soft scoring
    """
    
    def __init__(self):
        self.model = embedding_model
    
    def calculate_text_similarity(self, text1: str, text2: str) -> float:
        """Calculate cosine similarity between two texts"""
        embeddings = self.model.encode([text1, text2])
        similarity = util.cos_sim(embeddings[0], embeddings[1])
        return float(similarity[0][0])
    
    def find_semantic_skill_matches(self, cv_skills: List[str], jd_skills: List[str], threshold: float = 0.6) -> Dict[str, List[Tuple[str, float]]]:
        """Find semantic matches between CV and JD skills"""
        matches = {}
        
        if not cv_skills or not jd_skills:
            return matches
        
        # Encode all skills
        cv_embeddings = self.model.encode(cv_skills)
        jd_embeddings = self.model.encode(jd_skills)
        
        # For each JD skill, find top CV matches
        for i, jd_skill in enumerate(jd_skills):
            similarities = util.cos_sim(jd_embeddings[i], cv_embeddings)[0]
            
            # Get matches above threshold
            skill_matches = []
            for j, sim in enumerate(similarities):
                if float(sim) >= threshold:
                    skill_matches.append((cv_skills[j], float(sim)))
            
            # Sort by similarity
            skill_matches.sort(key=lambda x: x[1], reverse=True)
            
            if skill_matches:
                matches[jd_skill] = skill_matches[:3]  # Top 3 matches
        
        return matches
    
    def calculate_experience_relevance(self, cv_experience: List[Experience], jd_text: str) -> float:
        """Calculate semantic relevance of experience to JD"""
        if not cv_experience:
            return 0.0
        
        # Combine all experience descriptions
        cv_exp_text = " ".join([
            f"{exp.title}. {' '.join(exp.description)}" 
            for exp in cv_experience
        ])
        
        # Extract responsibilities from JD
        resp_section = re.search(r'responsibilities?[:\n](.*?)(?=requirements|qualifications|$)', 
                                 jd_text, re.IGNORECASE | re.DOTALL)
        
        if resp_section:
            jd_resp = resp_section.group(1)
        else:
            jd_resp = jd_text[:500]  # Use first 500 chars as fallback
        
        # Calculate similarity
        similarity = self.calculate_text_similarity(cv_exp_text, jd_resp)
        return similarity
    
    def analyze(self, parsed_cv: ParsedCV, job_req: JobRequirements) -> SemanticAnalysis:
        """Perform semantic analysis"""
        print("🧠 Performing Semantic Analysis...")
        
        # Overall CV-JD similarity
        cv_jd_sim = self.calculate_text_similarity(
            parsed_cv.raw_text[:1000],  # First 1000 chars
            job_req.raw_text[:1000]
        )
        
        # Semantic skill matching
        all_jd_skills = job_req.must_have_skills + job_req.nice_to_have_skills
        skill_matches = self.find_semantic_skill_matches(
            parsed_cv.skills, 
            all_jd_skills
        )
        
        # Experience relevance
        exp_relevance = self.calculate_experience_relevance(
            parsed_cv.experience,
            job_req.raw_text
        )
        
        analysis = SemanticAnalysis(
            cv_jd_similarity=round(cv_jd_sim, 3),
            skill_semantic_matches=skill_matches,
            experience_relevance=round(exp_relevance, 3)
        )
        
        print(f"✅ CV-JD Similarity: {analysis.cv_jd_similarity:.2%}")
        print(f"✅ Found {len(skill_matches)} semantic skill matches")
        print(f"✅ Experience Relevance: {analysis.experience_relevance:.2%}")
        
        return analysis

print("✅ Semantic Similarity Agent initialized")

✅ Semantic Similarity Agent initialized


## 🤖 Agent 5: Formatting & ATS Compatibility Agent (Validator Agent)

In [8]:
class FormattingValidatorAgent:
    """
    Agent Type: Validator / Inspector Agent
    Role: Detect ATS-unfriendly formatting
    Characteristics: Rule-based, checklist-driven, flags risks
    """
    
    def check_for_tables(self, file_path: str) -> bool:
        """Check if document contains tables"""
        if file_path.endswith('.docx'):
            try:
                doc = docx.Document(file_path)
                return len(doc.tables) > 0
            except:
                return False
        return False
    
    def check_for_columns(self, text: str) -> bool:
        """Detect multi-column layout (heuristic)"""
        # If there are many lines with similar short length, might be columns
        lines = text.split('\n')
        short_lines = [l for l in lines if 20 < len(l) < 40]
        
        # If >30% of lines are short and similar, likely columns
        if len(short_lines) / len(lines) > 0.3:
            return True
        return False
    
    def check_for_images(self, file_path: str) -> bool:
        """Check for embedded images"""
        if file_path.endswith('.docx'):
            try:
                doc = docx.Document(file_path)
                # Check for images in document
                for rel in doc.part.rels.values():
                    if "image" in rel.target_ref:
                        return True
            except:
                pass
        return False
    
    def check_for_icons(self, text: str) -> bool:
        """Detect special characters/icons"""
        # Unicode ranges for common icons
        icon_ranges = [
            (0x2600, 0x26FF),  # Miscellaneous Symbols
            (0x2700, 0x27BF),  # Dingbats
            (0x1F300, 0x1F9FF),  # Emoji
        ]
        
        for char in text:
            code = ord(char)
            for start, end in icon_ranges:
                if start <= code <= end:
                    return True
        return False
    
    def check_section_headers(self, text: str) -> List[str]:
        """Check for standard section headers"""
        standard_sections = [
            'experience', 'education', 'skills', 'summary',
            'objective', 'certifications', 'projects'
        ]
        
        missing_sections = []
        text_lower = text.lower()
        
        for section in standard_sections:
            if section not in text_lower:
                missing_sections.append(section)
        
        return missing_sections
    
    def validate(self, file_path: str, parsed_cv: ParsedCV) -> FormatIssues:
        """Validate ATS compatibility"""
        print("🔍 Checking ATS Compatibility...")
        
        has_tables = self.check_for_tables(file_path)
        has_columns = self.check_for_columns(parsed_cv.raw_text)
        has_images = self.check_for_images(file_path)
        has_icons = self.check_for_icons(parsed_cv.raw_text)
        
        parsing_risks = []
        recommendations = []
        
        if has_tables:
            parsing_risks.append("Tables detected - may cause parsing errors")
            recommendations.append("Remove tables and use simple lists instead")
        
        if has_columns:
            parsing_risks.append("Multi-column layout detected")
            recommendations.append("Use single-column layout for better ATS parsing")
        
        if has_images:
            parsing_risks.append("Images/graphics detected")
            recommendations.append("Remove images - ATS cannot read them")
        
        if has_icons:
            parsing_risks.append("Special characters/icons detected")
            recommendations.append("Replace icons with text (e.g., use 'Email:' instead of ✉)")
        
        missing_sections = self.check_section_headers(parsed_cv.raw_text)
        if 'experience' in missing_sections or 'education' in missing_sections:
            parsing_risks.append("Missing standard section headers")
            recommendations.append("Add clear section headers: EXPERIENCE, EDUCATION, SKILLS")
        
        ats_friendly = len(parsing_risks) == 0
        
        if ats_friendly:
            recommendations.append("Resume is ATS-friendly! ✅")
        
        issues = FormatIssues(
            has_tables=has_tables,
            has_columns=has_columns,
            has_images=has_images,
            has_icons=has_icons,
            parsing_risks=parsing_risks,
            ats_friendly=ats_friendly,
            recommendations=recommendations
        )
        
        if ats_friendly:
            print("✅ Resume is ATS-friendly!")
        else:
            print(f"⚠️  Found {len(parsing_risks)} ATS compatibility issues")
        
        return issues

print("✅ Formatting Validator Agent initialized")

✅ Formatting Validator Agent initialized


## 🤖 Agent 6: Bias & Compliance Agent (Policy/Safety Agent)

In [9]:
class BiasComplianceAgent:
    """
    Agent Type: Policy / Safety Agent
    Role: Ensure ethical and legal safety
    Characteristics: Conservative, keyword detection, binary flags
    """
    
    def __init__(self):
        # Age indicators
        self.age_keywords = [
            r'\b(19|20)\d{2}\s*born\b',  # Year of birth
            r'\bage\s*:\s*\d+\b',  # "Age: 25"
            r'\b\d+\s*years\s*old\b',  # "25 years old"
        ]
        
        # Gender indicators (pronouns, gender-specific terms)
        self.gender_keywords = [
            r'\b(he|she|his|her|him)\b',
            r'\b(male|female|man|woman)\b',
        ]
        
        # Nationality/ethnicity indicators
        self.nationality_keywords = [
            r'\bnationality\s*:',
            r'\bethnicity\s*:',
            r'\brace\s*:',
            r'\bcitizenship\s*:',
        ]
    
    def detect_age_indicators(self, text: str) -> List[str]:
        """Detect age-related information"""
        findings = []
        text_lower = text.lower()
        
        for pattern in self.age_keywords:
            matches = re.findall(pattern, text_lower, re.IGNORECASE)
            if matches:
                findings.extend(matches)
        
        return findings
    
    def detect_gender_bias(self, text: str) -> List[str]:
        """Detect gender-related information"""
        findings = []
        text_lower = text.lower()
        
        for pattern in self.gender_keywords:
            matches = re.findall(pattern, text_lower, re.IGNORECASE)
            if matches:
                findings.extend(matches)
        
        return findings
    
    def detect_nationality_references(self, text: str) -> List[str]:
        """Detect nationality/ethnicity references"""
        findings = []
        text_lower = text.lower()
        
        for pattern in self.nationality_keywords:
            matches = re.findall(pattern, text_lower, re.IGNORECASE)
            if matches:
                findings.append(pattern)
        
        return findings
    
    def detect_photo(self, file_path: str) -> bool:
        """Detect if resume contains a photo"""
        if file_path.endswith('.docx'):
            try:
                doc = docx.Document(file_path)
                for rel in doc.part.rels.values():
                    if "image" in rel.target_ref.lower():
                        # Check if it's likely a headshot (heuristic)
                        return True
            except:
                pass
        return False
    
    def check_compliance(self, file_path: str, parsed_cv: ParsedCV) -> BiasFlags:
        """Check for bias and compliance issues"""
        print("⚖️  Checking Bias & Compliance...")
        
        age_indicators = self.detect_age_indicators(parsed_cv.raw_text)
        gender_bias = self.detect_gender_bias(parsed_cv.raw_text)
        nationality_refs = self.detect_nationality_references(parsed_cv.raw_text)
        photo_detected = self.detect_photo(file_path)
        
        compliance_issues = []
        
        if age_indicators:
            compliance_issues.append("Age information detected - remove to avoid age discrimination")
        
        if gender_bias:
            compliance_issues.append("Gender-specific language detected - use gender-neutral terms")
        
        if nationality_refs:
            compliance_issues.append("Nationality/ethnicity references detected - remove unless required")
        
        if photo_detected:
            compliance_issues.append("Photo detected - remove to avoid unconscious bias")
        
        is_compliant = len(compliance_issues) == 0
        
        flags = BiasFlags(
            age_indicators=age_indicators,
            gender_bias=gender_bias,
            nationality_references=nationality_refs,
            photo_detected=photo_detected,
            compliance_issues=compliance_issues,
            is_compliant=is_compliant
        )
        
        if is_compliant:
            print("✅ No compliance issues detected")
        else:
            print(f"⚠️  Found {len(compliance_issues)} compliance issues")
        
        return flags

print("✅ Bias & Compliance Agent initialized")

✅ Bias & Compliance Agent initialized


## 🤖 Agent 7: Feedback & Rewrite Agent (Generative Coach Agent)

In [10]:
import re
from typing import List, Dict
from sklearn.metrics.pairwise import cosine_similarity

class FeedbackAgent:
    """
    Agent Type: Generative Coach
    Role: Suggests REAL content (bullet points) for missing skills.
    """
    
    def __init__(self):
        self.model = embedding_model 
        
        # 1. KNOWLEDGE BASE: Pre-written high-impact bullet points for common skills
        # This allows the agent to give "Real Suggestions" you can copy-paste.
        self.skill_content_db = {
            'python': "Developed automated scripts using Python (Pandas/NumPy) to process data, reducing manual effort by 40%.",
            'java': "Architected scalable backend microservices using Java Spring Boot, handling 10k+ concurrent requests.",
            'react': "Designed responsive frontend interfaces using React.js and Redux, improving user engagement time by 25%.",
            'node': "Built RESTful APIs using Node.js/Express, optimizing database queries for 50% faster response times.",
            'node.js': "Built RESTful APIs using Node.js/Express, optimizing database queries for 50% faster response times.",
            'kubernetes': "Orchestrated containerized applications using Kubernetes (K8s), ensuring 99.9% uptime for production services.",
            'docker': "Containerized legacy applications using Docker, reducing deployment cycle time from 2 days to 30 minutes.",
            'aws': "Deployed fault-tolerant infrastructure on AWS (EC2, S3, Lambda), reducing monthly cloud costs by 15%.",
            'cloud': "Managed cloud infrastructure migration, ensuring zero downtime and 100% data integrity.",
            'sql': "Optimized complex SQL queries and stored procedures, improving report generation speed by 60%.",
            'machine learning': "Trained predictive models using Scikit-Learn/TensorFlow, achieving 92% accuracy in forecasting.",
            'spark': "Engineered real-time data pipelines using Apache Spark, processing TB-scale datasets for analytics.",
            'ci/cd': "Implemented CI/CD pipelines using Jenkins/GitLab CI, enabling daily production releases.",
            'agile': "Led daily stand-ups and sprint planning in an Agile/Scrum environment, improving team velocity by 20%."
        }

        self.action_verbs = ['Developed', 'Engineered', 'Deployed', 'Architected', 'Automated', 'Optimized', 'Led', 'Managed']

    def generate_content_suggestion(self, missing_skill: str) -> str:
        """Finds or generates a bullet point for a missing skill"""
        missing_lower = missing_skill.lower()
        
        # 1. Direct Knowledge Base Lookup
        for key, bullet in self.skill_content_db.items():
            if key in missing_lower or missing_lower in key:
                return bullet
                
        # 2. Generic Fallback (if skill not in DB)
        return f"Integrated {missing_skill} into the development workflow to enhance system performance and scalability."

    def detect_fluff(self, bullets: List[str], jd_text: str) -> List[str]:
        if not bullets or not jd_text: return []
        try:
            jd_emb = self.model.encode(jd_text).reshape(1, -1)
            bullet_embs = self.model.encode(bullets)
            scores = cosine_similarity(bullet_embs, jd_emb).flatten()
            return [bullets[i] for i, score in enumerate(scores) if score < 0.15 and len(bullets[i]) > 30]
        except:
            return []

    def analyze_bullet_strength(self, bullet: str) -> str:
        words = bullet.lower().split()
        if not words or words[0] not in [v.lower() for v in self.action_verbs]:
            return "Start with an Action Verb"
        if not re.search(r'\d+%|\$\d+|\d+x|\d+\+', bullet):
            return "Add a Metric (e.g. 20%)"
        return "Strong"

    def generate_feedback(self, parsed_cv: ParsedCV, job_req: JobRequirements, ats_score: ATSScore) -> Feedback:
        print("💡 Feedback Agent: Generating REAL content suggestions...")
        
        priorities = []
        stronger_bullets = []
        
        # 1. REAL SUGGESTIONS for Missing Skills
        if ats_score.missing_must_have:
            priorities.append(f"🔴 CRITICAL: Missing Keywords: {', '.join(ats_score.missing_must_have[:3])}")
            
            for skill in ats_score.missing_must_have[:3]: # Suggest for top 3 missing
                suggestion = self.generate_content_suggestion(skill)
                # Add concrete advice to the list
                priorities.append(f"   💡 Suggestion for '{skill}': Add a bullet like -> \"{suggestion}\"")

        # 2. Fix Weak Bullets
        all_bullets = [b for exp in parsed_cv.experience for b in exp.description]
        for bullet in all_bullets[:5]:
            issue = self.analyze_bullet_strength(bullet)
            if issue != "Strong":
                stronger_bullets.append(f"Rewrite '{bullet[:30]}...' -> {issue}")

        return Feedback(
            missing_keywords=ats_score.missing_must_have,
            skill_rewording={},
            stronger_bullets=stronger_bullets,
            improvement_priority=priorities,
            rewritten_sections={}
        )

print("✅ Feedback Agent Updated: Now Generates Real Content Suggestions")

✅ Feedback Agent Updated: Now Generates Real Content Suggestions


In [11]:
import json
from dataclasses import asdict

def save_report_json(report, filename: str):
    """Saves the FinalReport object to a JSON file."""
    try:
        # Convert dataclass to dict if it isn't already
        if hasattr(report, '__dataclass_fields__'):
            data = asdict(report)
        else:
            data = report
            
        with open(filename, 'w', encoding='utf-8') as f:
            json.dump(data, f, indent=4, default=str)
            
        print(f"✅ Report successfully saved to: {filename}")
    except Exception as e:
        print(f"❌ Error saving JSON: {e}")

def print_final_report(report):
    """Prints a formatted summary of the analysis to the console."""
    
    # Handle both object and dict access
    def get_attr(obj, key):
        if isinstance(obj, dict):
            return obj.get(key)
        return getattr(obj, key, None)

    decision = get_attr(report, 'decision')
    score = get_attr(get_attr(report, 'ats_score'), 'overall_score')
    summary = get_attr(report, 'summary')
    
    print("\n" + "="*60)
    print("📋 FINAL ATS ANALYSIS REPORT")
    print("="*60)
    
    # 1. Executive Summary
    print(f"\n🔹 DECISION: {decision}")
    if score:
        print(f"🔹 OVERALL SCORE: {score:.1f}/100")
    print(f"\n📝 SUMMARY:\n{summary}")
    
    # 2. Missing Skills (if any)
    ats_score = get_attr(report, 'ats_score')
    missing = get_attr(ats_score, 'missing_must_have')
    if missing:
        print(f"\n⚠️ MISSING CRITICAL SKILLS:\n" + ", ".join(missing))
    
    # 3. Improvement Checklist
    checklist = get_attr(report, 'improvement_checklist')
    if checklist:
        print("\n✅ IMPROVEMENT CHECKLIST:")
        for item in checklist:
            print(f"  - {item}")
            
    # 4. Visual Layout (if available)
    visual = get_attr(report, 'visual_report')
    if visual and isinstance(visual, dict):
        pos = visual.get('skills_position', 'Unknown')
        print(f"\n👁️ VISUAL LAYOUT:\n  - Skills Position: {pos}")
        if visual.get('issues'):
            print(f"  - Issues: {', '.join(visual['issues'])}")

    print("\n" + "="*60 + "\n")

## 🤖 Auto-Improve 

In [12]:
!pip install pdfplumber

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.9/67.9 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.0/60.0 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 66.1 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 65.7 MB/s eta 0:00:00:00:01


In [13]:
import pdfplumber
import statistics

class VisualLayoutAgent:
    """
    Agent Type: Visual Analyst
    Role: Inspects physical layout, font usage, and section placement.
    """
    
    def analyze_layout(self, file_path: str):
        print("👁️ Visual Agent: Scanning document layout...")
        
        report = {
            "font_names": set(),
            "font_sizes": [],
            "skills_position": "Not Found", # Top, Middle, Bottom, or Not Found
            "layout_score": 100,
            "issues": []
        }
        
        try:
            with pdfplumber.open(file_path) as pdf:
                full_text_height = 0
                skills_y_coord = None
                total_chars = 0
                
                # Check Page 1 (Critical for ATS)
                first_page = pdf.pages[0]
                height = first_page.height
                width = first_page.width
                
                # Extract words with layout info
                words = first_page.extract_words(extra_attrs=["fontname", "size", "top"])
                
                for word in words:
                    # Collect Font Stats
                    report["font_names"].add(word["fontname"])
                    report["font_sizes"].append(float(word["size"]))
                    total_chars += len(word["text"])
                    
                    # Check for "Skills" section header
                    if "skill" in word["text"].lower() and float(word["size"]) > 11:
                        # Ensure it's a header (usually larger or standalone)
                        skills_y_coord = word["top"]

                # 1. Analyze Skills Placement
                if skills_y_coord:
                    # 0 is top, 'height' is bottom
                    relative_pos = skills_y_coord / height
                    if relative_pos < 0.3:
                        report["skills_position"] = "Top (Excellent)"
                    elif relative_pos < 0.6:
                        report["skills_position"] = "Middle (Good)"
                    else:
                        report["skills_position"] = "Bottom (Risk)"
                        report["issues"].append("Visual: 'Skills' section is too low. Move it to the top 30% of page 1.")
                        report["layout_score"] -= 15
                else:
                    report["issues"].append("Visual: Could not visually locate a distinct 'Skills' header.")

                # 2. Analyze Font Sizes
                if report["font_sizes"]:
                    avg_size = statistics.mean(report["font_sizes"])
                    min_size = min(report["font_sizes"])
                    
                    if min_size < 9:
                        report["issues"].append(f"Visual: Text is too small ({min_size}pt). Minimum recommended is 10pt.")
                        report["layout_score"] -= 10
                    
                    if len(report["font_names"]) > 3:
                        report["issues"].append(f"Visual: Too many font types detected ({len(report['font_names'])}). Keep it to 1 or 2.")
                        report["layout_score"] -= 5

                # 3. Analyze Clutter (Density)
                # A standard readable page has ~2500-3000 chars max.
                if total_chars > 3500:
                    report["issues"].append("Visual: Page is too dense/cluttered. Add more whitespace.")
                    report["layout_score"] -= 10

        except Exception as e:
            print(f"Visual Analysis Warning: {e}")
            report["issues"].append("Could not perform visual analysis (file might not be a text-PDF).")
            
        return report

## 🤖 Agent 8: Orchestrator Agent (Manager Agent)

In [14]:
class OrchestratorAgent:
    """
    Agent Type: Orchestrator / Manager Agent
    Role: Coordinate all agents and produce final report
    Characteristics: High-level reasoning, conflict resolution, final decision
    """
    
    def __init__(self):
        self.cv_parser = CVParsingAgent()
        self.jd_analyzer = JobDescriptionAgent()
        self.ats_scorer = ATSScoringAgent()
        self.semantic_analyzer = SemanticSimilarityAgent()
        self.format_validator = FormattingValidatorAgent()
        self.bias_checker = BiasComplianceAgent()
        self.feedback_agent = FeedbackAgent()
        
    
    def determine_decision(self, ats_score: ATSScore, semantic_analysis: SemanticAnalysis,
                          format_issues: FormatIssues, bias_flags: BiasFlags) -> Decision:
        """Determine final pass/borderline/reject decision"""
        
        # Automatic reject conditions
        if not bias_flags.is_compliant:
            return Decision.REJECT
        
        if len(ats_score.missing_must_have) > 3:
            return Decision.REJECT
        
        # Pass conditions
        if (
            ats_score.overall_score >= 75 and
            semantic_analysis.cv_jd_similarity >= 0.6 and
            format_issues.ats_friendly
        ):
            return Decision.PASS
        
        # Borderline conditions
        if (
            ats_score.overall_score >= 60 or
            semantic_analysis.cv_jd_similarity >= 0.5
        ):
            return Decision.BORDERLINE
        
        # Default to reject
        return Decision.REJECT
    
    def create_improvement_checklist(self, feedback: Feedback, format_issues: FormatIssues,
                                    bias_flags: BiasFlags) -> List[str]:
        """Create actionable improvement checklist"""
        checklist = []
        
        # From feedback
        checklist.extend(feedback.improvement_priority)
        
        # From format issues
        if not format_issues.ats_friendly:
            checklist.extend([f"🟠 {rec}" for rec in format_issues.recommendations])
        
        # From bias/compliance
        if not bias_flags.is_compliant:
            checklist.extend([f"🔴 {issue}" for issue in bias_flags.compliance_issues])
        
        return checklist
    
    def generate_summary(self, decision: Decision, ats_score: ATSScore,
                        semantic_analysis: SemanticAnalysis) -> str:
        """Generate executive summary"""
        summaries = {
            Decision.PASS: f"✅ **PASS** - Strong candidate with {ats_score.overall_score:.0f}/100 ATS score. "
                          f"Resume shows {semantic_analysis.cv_jd_similarity:.0%} semantic match with job requirements. "
                          f"Proceed to interview.",
            
            Decision.BORDERLINE: f"⚠️  **BORDERLINE** - Moderate fit with {ats_score.overall_score:.0f}/100 ATS score. "
                                f"Resume shows {semantic_analysis.cv_jd_similarity:.0%} match. "
                                f"Consider with improvements or manual review.",
            
            Decision.REJECT: f"❌ **REJECT** - Insufficient match with {ats_score.overall_score:.0f}/100 ATS score. "
                            f"Resume shows {semantic_analysis.cv_jd_similarity:.0%} match. "
                            f"Candidate does not meet minimum requirements."
        }
        
        return summaries[decision]
    
    def process(self, cv_file_path: str, jd_text: str) -> FinalReport:
        """
        Main orchestration method - coordinates all agents
        """
        print("\n" + "="*70)
        print("🚀 STARTING ATS MULTI-AGENT ANALYSIS")
        print("="*70 + "\n")
        
        # Step 1: Parse CV
        print("[1/8] Running CV Parsing Agent...")
        parsed_cv = self.cv_parser.parse(cv_file_path)
        
        # Step 2: Analyze JD
        print("\n[2/8] Running Job Description Agent...")
        job_req = self.jd_analyzer.analyze(jd_text)
        
        # Step 3: ATS Scoring
        print("\n[3/8] Running ATS Scoring Agent...")
        ats_score = self.ats_scorer.score(parsed_cv, job_req)
        
        # Step 4: Semantic Analysis
        print("\n[4/8] Running Semantic Similarity Agent...")
        semantic_analysis = self.semantic_analyzer.analyze(parsed_cv, job_req)
        
        # Step 5: Format Validation
        print("\n[5/8] Running Formatting Validator Agent...")
        format_issues = self.format_validator.validate(cv_file_path, parsed_cv)
        
        # Step 6: Bias Check
        print("\n[6/8] Running Bias & Compliance Agent...")
        bias_flags = self.bias_checker.check_compliance(cv_file_path, parsed_cv)
        
        # Step 7: Generate Feedback
        print("\n[7/8] Running Feedback Agent...")
        feedback = self.feedback_agent.generate_feedback(parsed_cv, job_req, ats_score)
        
        # Step 8: Make Final Decision
        print("\n[8/8] Making Final Decision...")
        decision = self.determine_decision(ats_score, semantic_analysis, format_issues, bias_flags)
        improvement_checklist = self.create_improvement_checklist(feedback, format_issues, bias_flags)
        summary = self.generate_summary(decision, ats_score, semantic_analysis)
        
        # Create final report
        report = FinalReport(
            decision=decision,
            ats_score=ats_score,
            semantic_analysis=semantic_analysis,
            format_issues=format_issues,
            bias_flags=bias_flags,
            feedback=feedback,
            improvement_checklist=improvement_checklist,
            summary=summary
        )
        
        print("\n" + "="*70)
        print("✅ ANALYSIS COMPLETE")
        print("="*70)
        
        return report

print("✅ Orchestrator Agent initialized")

✅ Orchestrator Agent initialized


## 📊 Report Generator & Visualization

In [15]:
class OrchestratorAgent:
    """
    Agent Type: Orchestrator / Manager Agent
    Role: Coordinate all agents (Text + Visual) and produce final report
    """
    
    def __init__(self):
        # 1. Text Analysis Agents
        self.cv_parser = CVParsingAgent()
        self.jd_analyzer = JobDescriptionAgent()
        self.ats_scorer = ATSScoringAgent()
        self.semantic_analyzer = SemanticSimilarityAgent()
        self.format_validator = FormattingValidatorAgent()
        self.bias_checker = BiasComplianceAgent()
        self.feedback_agent = FeedbackAgent()
        
        # 2. Visual Analysis Agent
        self.visual_agent = VisualLayoutAgent()
    
    def determine_decision(self, ats_score, semantic_analysis, format_issues, bias_flags, visual_report) -> Decision:
        """Determine final pass/borderline/reject decision"""
        
        # Automatic reject conditions
        if not bias_flags.is_compliant:
            return Decision.REJECT
        
        # Reject if skills are buried at the bottom (Visual Check - only if available)
        if visual_report.get('skills_position', '').startswith('Bottom'):
            return Decision.BORDERLINE
        
        score = ats_score.overall_score
        
        # Pass conditions
        if (score >= 75 and semantic_analysis.cv_jd_similarity >= 0.6 and format_issues.ats_friendly):
            return Decision.PASS
        
        # Borderline conditions
        if (score >= 60 or semantic_analysis.cv_jd_similarity >= 0.5):
            return Decision.BORDERLINE
        
        return Decision.REJECT
    
    def create_improvement_checklist(self, feedback, format_issues, bias_flags, visual_report) -> List[str]:
        """Create actionable improvement checklist combining ALL agents"""
        checklist = []
        
        # 1. Content Feedback
        checklist.extend(feedback.improvement_priority)
        
        # 2. Visual Layout Issues
        if visual_report.get('issues'):
            for issue in visual_report['issues']:
                checklist.append(f"🎨 {issue}")
        
        # 3. Formatting
        if not format_issues.ats_friendly:
            checklist.extend([f"🟠 {rec}" for rec in format_issues.recommendations])
        
        # 4. Compliance
        if not bias_flags.is_compliant:
            checklist.extend([f"🔴 {issue}" for issue in bias_flags.compliance_issues])
        
        return checklist
    
    def process(self, cv_file_path: str, jd_text: str) -> FinalReport:
        print("\n" + "="*70)
        print("🚀 STARTING ATS MULTI-AGENT ANALYSIS")
        print("="*70 + "\n")
        
        # Step 1: Parse CV
        print("[1/9] 📄 Parsing CV...")
        parsed_cv = self.cv_parser.parse(cv_file_path)
        
        # Step 2: Analyze JD
        print("[2/9] 📋 Analyzing JD...")
        job_req = self.jd_analyzer.analyze(jd_text)
        
        # Step 3: ATS Scoring
        print("[3/9] 📊 ATS Scoring...")
        ats_score = self.ats_scorer.score(parsed_cv, job_req)
        
        # Step 4: Semantic Analysis
        print("[4/9] 🧠 Semantic Analysis...")
        semantic_analysis = self.semantic_analyzer.analyze(parsed_cv, job_req)
        
        # Step 5: Format Validation
        print("[5/9] 📐 Format Validation...")
        format_issues = self.format_validator.validate(cv_file_path, parsed_cv)
        
        # Step 6: Bias Check
        print("[6/9] ⚖️  Compliance Check...")
        bias_flags = self.bias_checker.check_compliance(cv_file_path, parsed_cv)
        
        # Step 7: Feedback (Fluff & Content)
        print("[7/9] 💡 Generating Content Feedback...")
        feedback = self.feedback_agent.generate_feedback(parsed_cv, job_req, ats_score)

        # Step 8: Visual Analysis (Safe Mode)
        print("[8/9] 👁️ Visual Layout Analysis...")
        visual_report = {"skills_position": "Skipped (Not PDF)", "issues": [], "layout_score": 100}
        
        if cv_file_path.lower().endswith(".pdf"):
            try:
                visual_report = self.visual_agent.analyze_layout(cv_file_path)
            except Exception as e:
                print(f"      ⚠️ Visual Agent failed gracefully: {e}")
        else:
            print("      ⚠️ Skipping Visual Analysis (File is not PDF)")
        
        # Step 9: Make Decision & Report
        print("[9/9] Finalizing Report...")
        decision = self.determine_decision(ats_score, semantic_analysis, format_issues, bias_flags, visual_report)
        checklist = self.create_improvement_checklist(feedback, format_issues, bias_flags, visual_report)
        
        # Generate Summary String
        # FIX: Added .value to access the string inside the Enum
        summary = (f"**{decision.value.upper()}** - Score: {ats_score.overall_score:.0f}/100. "
                   f"Visual Layout: {visual_report.get('skills_position', 'Unknown')}. "
                   f"Semantic Match: {semantic_analysis.cv_jd_similarity:.0%}")

        # Create final report object
        report = FinalReport(
            decision=decision.value, # Return string value for JSON serialization
            ats_score=ats_score,
            semantic_analysis=semantic_analysis,
            format_issues=format_issues,
            bias_flags=bias_flags,
            feedback=feedback,
            improvement_checklist=checklist,
            summary=summary
        )
        
        # Attach visual_report dynamically
        report.visual_report = visual_report 

        print("\n" + "="*70)
        print("✅ ANALYSIS COMPLETE")
        print("="*70)
        
        return report

print("✅ Orchestrator Agent Fixed (PDF Safety Check + Enum Fix)")

✅ Orchestrator Agent Fixed (PDF Safety Check + Enum Fix)


## 🧪 Example Usage

In [16]:
# Create sample CV text file for testing
sample_cv = """John Doe
Email: john.doe@email.com
Phone: +1-555-0123

SUMMARY
Experienced Software Engineer with 5 years in machine learning and backend development.

SKILLS
Python, Java, Machine Learning, TensorFlow, PyTorch, SQL, AWS, Docker, Git

EXPERIENCE

Senior Software Engineer
Tech Company Inc.
2021 - Present
• Developed machine learning models for customer segmentation
• Implemented REST APIs using Python and Flask
• Improved model accuracy by 15% through feature engineering
• Led team of 3 junior developers

Software Engineer
StartUp Co.
2019 - 2021
• Built data pipelines processing 1M+ records daily
• Deployed models to production using Docker and AWS
• Collaborated with cross-functional teams

EDUCATION

Bachelor of Science in Computer Science
State University
2019
GPA: 3.8/4.0
"""

# Save sample CV
with open('sample_cv.txt', 'w') as f:
    f.write(sample_cv)

# Sample Job Description
sample_jd = """Machine Learning Engineer

We are looking for an experienced Machine Learning Engineer to join our team.

REQUIREMENTS:
• 3+ years of experience in machine learning
• Required: Python, TensorFlow or PyTorch, SQL
• Required: Experience with cloud platforms (AWS, GCP, or Azure)
• Preferred: Deep Learning, NLP, Computer Vision
• Preferred: Docker, Kubernetes
• Strong communication and teamwork skills

RESPONSIBILITIES:
• Design and implement machine learning models
• Deploy models to production
• Collaborate with data scientists and engineers
• Optimize model performance
"""

print("✅ Sample data created")

✅ Sample data created


In [17]:
# Initialize orchestrator
orchestrator = OrchestratorAgent()

# Process CV and JD
report = orchestrator.process(
    cv_file_path='sample_cv.txt',
    jd_text=sample_jd
)


🚀 STARTING ATS MULTI-AGENT ANALYSIS

[1/9] 📄 Parsing CV...
🔍 Parsing CV: sample_cv.txt
✅ Extracted 13 skills, 1 experiences, 1 education entries
[2/9] 📋 Analyzing JD...
🔍 Analyzing Job Description with Semantic Agent...
[3/9] 📊 ATS Scoring...
📊 Calculating ATS Score (Semantic Title Mode)...
[4/9] 🧠 Semantic Analysis...
🧠 Performing Semantic Analysis...
✅ CV-JD Similarity: 61.30%
✅ Found 6 semantic skill matches
✅ Experience Relevance: 53.80%
[5/9] 📐 Format Validation...
🔍 Checking ATS Compatibility...
✅ Resume is ATS-friendly!
[6/9] ⚖️  Compliance Check...
⚖️  Checking Bias & Compliance...
✅ No compliance issues detected
[7/9] 💡 Generating Content Feedback...
💡 Feedback Agent: Generating REAL content suggestions...
[8/9] 👁️ Visual Layout Analysis...
      ⚠️ Skipping Visual Analysis (File is not PDF)
[9/9] Finalizing Report...

✅ ANALYSIS COMPLETE


In [18]:
# Print final report
print_final_report(report)


📋 FINAL ATS ANALYSIS REPORT

🔹 DECISION: borderline
🔹 OVERALL SCORE: 57.6/100

📝 SUMMARY:
**BORDERLINE** - Score: 58/100. Visual Layout: Skipped (Not PDF). Semantic Match: 61%

⚠️ MISSING CRITICAL SKILLS:
azure, gcp

✅ IMPROVEMENT CHECKLIST:
  - 🔴 CRITICAL: Missing Keywords: azure, gcp
  -    💡 Suggestion for 'azure': Add a bullet like -> "Integrated azure into the development workflow to enhance system performance and scalability."
  -    💡 Suggestion for 'gcp': Add a bullet like -> "Integrated gcp into the development workflow to enhance system performance and scalability."

👁️ VISUAL LAYOUT:
  - Skills Position: Skipped (Not PDF)




In [19]:
# Save report as JSON
save_report_json(report, 'ats_analysis_report.json')

✅ Report successfully saved to: ats_analysis_report.json


## 🚀 Advanced Features & Extensions

### Potential Enhancements:

1. **LLM Integration** (Optional)
   - Add Claude/GPT for better feedback generation
   - Improve semantic understanding
   
2. **Database Integration**
   - Store analysis results
   - Track improvements over time
   
3. **API Wrapper**
   - REST API for integration
   - Batch processing
   
4. **Web Interface**
   - Streamlit/Gradio UI
   - Real-time analysis
   
5. **Industry-Specific Models**
   - Tech vs Finance vs Healthcare
   - Customized scoring weights
   
6. **Multi-Language Support**
   - Parse CVs in different languages
   - Translation capabilities

## 📚 Agent Communication Flow

```
CV (PDF/DOCX) ──────┐
                     │
                     ├──► [1] CV Parsing Agent
                     │         │
Job Description ─────┤         │
                     │         ├──► [2] JD Understanding Agent
                     │         │           │
                     └─────────┴───────────┤
                                           │
                        ┌──────────────────┴──────────────────┐
                        │                                      │
                  [3] ATS Scoring              [4] Semantic Similarity
                        │                                      │
                        │                                      │
                  [5] Format Validator          [6] Bias Checker
                        │                                      │
                        └──────────────────┬──────────────────┘
                                           │
                                  [7] Feedback Agent
                                           │
                                  [8] Orchestrator
                                           │
                                    Final Report
```

## 🎯 Production Deployment Checklist

- [ ] Add comprehensive error handling
- [ ] Implement logging (structured logs)
- [ ] Add monitoring and metrics
- [ ] Create configuration management
- [ ] Add authentication/authorization
- [ ] Implement rate limiting
- [ ] Add caching layer
- [ ] Write comprehensive tests
- [ ] Document API endpoints
- [ ] Set up CI/CD pipeline
- [ ] Add data privacy compliance (GDPR)
- [ ] Implement audit logging

In [20]:
pip install fastapi uvicorn pyngrok transformers==4.52.4 accelerate -q

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.5/10.5 MB 83.2 MB/s eta 0:00:00:00:01:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 57.3 MB/s eta 0:00:0000:01
Note: you may need to restart the kernel to use updated packages.


In [21]:
NGROK_TOKEN = "39RPzOZHvRI8YLFxpzoasI6WUmI_2m9PMaphjvsyPctU5o2uj"
API_KEY = "secret123"


In [ ]:
# ============================================================================
# PRODUCTION VERSION - No PDF Generation
# Replace Cell 19 with this code
# ============================================================================

import sys
import subprocess
import os
import asyncio
import tempfile
from pathlib import Path
from dataclasses import asdict
from typing import Optional, List

# 1. Install Dependencies
def install(package):
    subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", package])

packages = ['fastapi', 'uvicorn', 'pyngrok', 'python-multipart', 'nest-asyncio', 'reportlab']
for pkg in packages:
    try:
        __import__(pkg.replace('-', '_'))
    except ImportError:
        install(pkg)

import nest_asyncio
nest_asyncio.apply()

# 2. Imports
from fastapi import FastAPI, File, UploadFile, HTTPException, Depends, Header, Form
# from fastapi.responses import FileResponse # Removed download import
from fastapi.middleware.cors import CORSMiddleware
from pydantic import BaseModel
import uvicorn
from pyngrok import ngrok, conf

# ============================================================================
# LOAD AGENTS
# ============================================================================

print("\n🔍 Loading agents...")
try:
    # Ensure all agents are loaded from previous cells
    cv_parser = CVParsingAgent()
    jd_analyzer = JobDescriptionAgent()
    ats_scorer = ATSScoringAgent()
    semantic_analyzer = SemanticSimilarityAgent()
    format_validator = FormattingValidatorAgent()
    bias_checker = BiasComplianceAgent()
    feedback_agent = FeedbackAgent()
    # cv_generator = CVGeneratorAgent() # Removed Generator Agent
    print("✅ All 7 agents loaded successfully!\n")
except NameError as e:
    print(f"❌ Error: {e}")
    print("⚠️  Please run the Agent Definition cells (Cells 4-15) first!")
    raise

# ============================================================================
# CONFIGURATION
# ============================================================================

NGROK_TOKEN = "39RPzOZHvRI8YLFxpzoasI6WUmI_2m9PMaphjvsyPctU5o2uj" 
API_KEY = "secret123"

# ============================================================================
# ORCHESTRATOR
# ============================================================================

class ProductionOrchestrator:
    def __init__(self):
        self.cv_parser = cv_parser
        self.jd_analyzer = jd_analyzer
        self.ats_scorer = ats_scorer
        self.semantic_analyzer = semantic_analyzer
        self.format_validator = format_validator
        self.bias_checker = bias_checker
        self.feedback_agent = feedback_agent
        # self.cv_generator = cv_generator # Removed
        
    def process(self, cv_file_path: str, jd_text: str) -> dict:
        print(f"\n{'='*70}")
        print("🚀 STARTING ANALYSIS")
        print(f"{'='*70}\n")
        
        # 1. Parse CV
        print("[1/7] 📄 Parsing CV...")
        parsed_cv = self.cv_parser.parse(cv_file_path)
        
        # 2. Analyze JD
        print("[2/7] 📋 Analyzing JD...")
        job_req = self.jd_analyzer.analyze(jd_text)
        
        # 3. ATS Score
        print("[3/7] 📊 ATS Scoring...")
        ats_score = self.ats_scorer.score(parsed_cv, job_req)
        
        # 4. Semantic Analysis
        print("[4/7] 🧠 Semantic Analysis...")
        semantic_analysis = self.semantic_analyzer.analyze(parsed_cv, job_req)
        
        # 5. Format Validation
        print("[5/7] 📐 Format Validation...")
        format_issues = self.format_validator.validate(cv_file_path, parsed_cv)
        
        # 6. Compliance Check
        print("[6/7] ⚖️  Compliance Check...")
        bias_flags = self.bias_checker.check_compliance(cv_file_path, parsed_cv)
        
        # 7. Feedback Generation
        print("[7/7] 💡 Generating Feedback...")
        feedback = self.feedback_agent.generate_feedback(parsed_cv, job_req, ats_score)
        
        # Decision Logic
        score = ats_score.overall_score
        if not bias_flags.is_compliant:
            decision = Decision.REJECT
        elif score >= 80:
            decision = Decision.PASS
        elif score >= 65:
            decision = Decision.BORDERLINE
        else:
            decision = Decision.REJECT
            
        # Removed Step 8: Auto-Improvement (PDF Generation)

        # Construct Response
        checklist = list(feedback.improvement_priority)
        if not format_issues.ats_friendly:
            checklist.extend([f"🟠 {r}" for r in format_issues.recommendations])
        
        summary = (f"**{decision.value.upper()}** - Score: {score:.0f}/100. "
                   f"Semantic Match: {semantic_analysis.cv_jd_similarity:.0%}")
        
        return {
            "decision": decision.value,
            "overall_score": round(score, 2),
            "summary": summary,
            "ats_score": asdict(ats_score),
            "semantic_analysis": asdict(semantic_analysis),
            "format_issues": asdict(format_issues),
            "bias_flags": asdict(bias_flags),
            "feedback": asdict(feedback),
            "improvement_checklist": checklist
        }

orchestrator = ProductionOrchestrator()

# ============================================================================
# API DEFINITION
# ============================================================================

app = FastAPI(title="ATS AI System")

app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"],
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)

class AnalysisRequest(BaseModel):
    job_description: str
    cv_text: Optional[str] = None

class AnalysisResponse(BaseModel):
    decision: str
    overall_score: float
    summary: str
    ats_score: dict
    semantic_analysis: dict
    format_issues: dict
    bias_flags: dict
    feedback: dict
    improvement_checklist: List[str]

async def verify_api_key(authorization: str = Header(None)):
    if not authorization or authorization != f"Bearer {API_KEY}":
        raise HTTPException(status_code=401, detail="Invalid API key")
    return True

@app.get("/")
async def root():
    return {"status": "online", "system": "ATS Multi-Agent"}

# Removed /api/download-cv endpoint

@app.post("/api/analyze", response_model=AnalysisResponse)
async def analyze(request: AnalysisRequest, authenticated: bool = Depends(verify_api_key)):
    try:
        with tempfile.NamedTemporaryFile(mode='w', delete=False, suffix='.txt', encoding='utf-8') as tmp:
            tmp.write(request.cv_text)
            tmp_path = tmp.name
        
        try:
            return orchestrator.process(tmp_path, request.job_description)
        finally:
            os.unlink(tmp_path)
    except Exception as e:
        raise HTTPException(status_code=500, detail=str(e))

@app.post("/api/analyze-file", response_model=AnalysisResponse)
async def analyze_file(job_description: str = Form(...), cv_file: UploadFile = File(...), authenticated: bool = Depends(verify_api_key)):
    try:
        file_ext = Path(cv_file.filename).suffix.lower()
        if file_ext not in ['.pdf', '.docx', '.txt']:
            raise HTTPException(status_code=400, detail="Unsupported file type")
        
        with tempfile.NamedTemporaryFile(delete=False, suffix=file_ext) as tmp:
            content = await cv_file.read()
            tmp.write(content)
            tmp_path = tmp.name
        
        try:
            return orchestrator.process(tmp_path, job_description)
        finally:
            os.unlink(tmp_path)
    except Exception as e:
        raise HTTPException(status_code=500, detail=str(e))

# ============================================================================
# SERVER STARTUP
# ============================================================================

async def run_server():
    try:
        conf.get_default().auth_token = NGROK_TOKEN
        # Kill existing tunnels
        tunnels = ngrok.get_tunnels()
        for t in tunnels:
            ngrok.disconnect(t.public_url)
            
        public_url = ngrok.connect(8000, bind_tls=True)
        print("\n" + "="*70)
        print(f"🚀 SERVER ONLINE: {public_url.public_url}")
        print("="*70 + "\n")
        
        config = uvicorn.Config(app, host="0.0.0.0", port=8000, log_level="error")
        server = uvicorn.Server(config)
        await server.serve()
    except Exception as e:
        print(f"❌ Server Error: {e}")

try:
    loop = asyncio.get_event_loop()
except RuntimeError:
    loop = asyncio.new_event_loop()
    asyncio.set_event_loop(loop)

if __name__ == "__main__":
    loop.run_until_complete(run_server())

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 17.6 MB/s eta 0:00:00

🔍 Loading agents...
✅ All 7 agents loaded successfully!

                                                                                                    
🚀 SERVER ONLINE: https://abhorrently-threadless-reina.ngrok-free.dev


🚀 STARTING ANALYSIS

[1/7] 📄 Parsing CV...
🔍 Parsing CV: /tmp/tmpwu_d8_5e.docx
✅ Extracted 12 skills, 0 experiences, 2 education entries
[2/7] 📋 Analyzing JD...
🔍 Analyzing Job Description with Semantic Agent...
[3/7] 📊 ATS Scoring...
📊 Calculating ATS Score (Semantic Title Mode)...
[4/7] 🧠 Semantic Analysis...
🧠 Performing Semantic Analysis...
✅ CV-JD Similarity: 60.50%
✅ Found 5 semantic skill matches
✅ Experience Relevance: 0.00%
[5/7] 📐 Format Validation...
🔍 Checking ATS Compatibility...
✅ Resume is ATS-friendly!
[6/7] ⚖️  Compliance Check...
⚖️  Checking Bias & Compliance...
✅ No compliance issues detected
[7/7] 💡 Generating Feedback...
💡 Feedback Agent: Generating REAL content 